In [ ]:
from google.oauth2 import service_account
from google.cloud import aiplatform

credentials = service_account.Credentials.from_service_account_file(
    'alura.json'
)

In [ ]:
client = aiplatform.gapic.PredictionServiceClient(credentials=credentials)

In [ ]:
import base64
from google.cloud.aiplatform.gapic.schema import predict

def predict_image_classification_sample(
    project: str,
    endpoint_id: str,
    filename: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
    service_account_json: str = 'alura.json',
):
    # Cria as credenciais a partir do arquivo de chave de serviço.
    credentials = service_account.Credentials.from_service_account_file(
        service_account_json
    )

    # As opções do cliente requerem pontos finais de API regionais.
    client_options = {"api_endpoint": api_endpoint}

    # Inicializa o cliente que será usado para criar e enviar solicitações.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options, credentials=credentials)

    with open(filename, "rb") as f:
        file_content = f.read()

    # O formato de cada instância deve estar de acordo com o esquema de entrada de previsão do modelo implantado.
    encoded_content = base64.b64encode(file_content).decode("utf-8")
    instance = predict.instance.ImageClassificationPredictionInstance(
        content=encoded_content,
    ).to_value()
    instances = [instance]

    # Consulte gs://google-cloud-aiplatform/schema/predict/params/image_classification_1.0.0.yaml para o formato dos parâmetros.
    parameters = predict.params.ImageClassificationPredictionParams(
        confidence_threshold=0.5,
        max_predictions=5,
    ).to_value()
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )

    # Faz a previsão
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )

    # Processa a resposta
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))

# Exemplo de como chamar a função:
# predict_image_classification_sample(
#     project="your-project-id",
#     endpoint_id="your-endpoint-id",
#     filename="/caminho/para/imagem.jpg",
#     service_account_json="/caminho/para/sua-chave-servico.json"
# )


In [ ]:
predict_image_classification_sample(
    project="634587506752",
    endpoint_id="7670344847970009088",
    filename="Russula_1117.jpg",
    service_account_json="alura.json"
)

response
 deployed_model_id: 4638643844517199872
 prediction: {'ids': ['8470721418147921920'], 'confidences': [0.949509263], 'displayNames': ['Russula']}
